In [8]:
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
import itertools
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# 1. Configuração do caminho
database_path = 'database_50k'

print("Carregando bases de dados...")
# Lendo as bases principais
authors_df = pd.read_csv(f'{database_path}/authorships.csv')
works_df = pd.read_csv(f'{database_path}/works.csv', low_memory=False)
topics_df = pd.read_csv(f'{database_path}/topics.csv')

# 2. Merge e limpeza dos dados
print("Fazendo merge e limpeza dos dados...")
merged_df = authors_df.merge(
    works_df[['id', 'publication_date', 'title', 'abstract', 'language']], 
    left_on='work_id', right_on='id'
)
merged_df['publication_date'] = pd.to_datetime(merged_df['publication_date'], errors='coerce')
merged_df = merged_df.dropna(subset=['publication_date', 'author_id', 'title', 'abstract', 'language']).drop(columns=['id'])
merged_df = merged_df[merged_df['language'] == 'en']

# 3. Divisão Temporal (Treino = Passado, Teste = Futuro)
# Ordena por data de publicação e divide 80/20
unique_works = merged_df[['work_id', 'publication_date']].drop_duplicates().sort_values('publication_date')
split_idx = int(len(unique_works) * 0.8)

train_work_ids = set(unique_works.iloc[:split_idx]['work_id'])
test_work_ids = set(unique_works.iloc[split_idx:]['work_id'])

train_df = merged_df[merged_df['work_id'].isin(train_work_ids)]
test_df = merged_df[merged_df['work_id'].isin(test_work_ids)]

# Identificar os autores alvo (autores que aparecem no teste E que já existiam no treino)
autores_treino = set(train_df['author_id'])
autores_teste = set(test_df['author_id'])
autores_alvo = list(autores_treino.intersection(autores_teste))

print("\n=== RESUMO DO SPLIT ===")
print(f"Trabalhos no Treino (Passado): {len(train_work_ids)}")
print(f"Trabalhos no Teste (Futuro): {len(test_work_ids)}")
print(f"Total de Autores no Treino: {len(autores_treino)}")
print(f"Total de Autores no Teste: {len(autores_teste)}")
print(f"Total de Autores Alvo (para avaliar no teste): {len(autores_alvo)}")

Carregando bases de dados...
Fazendo merge e limpeza dos dados...

=== RESUMO DO SPLIT ===
Trabalhos no Treino (Passado): 19916
Trabalhos no Teste (Futuro): 4980
Total de Autores no Treino: 25262
Total de Autores no Teste: 11441
Total de Autores Alvo (para avaliar no teste): 2903


In [9]:
print("Construindo o grafo de coautorias do Treino (Passado)...")
grafo_treino = defaultdict(set)

# Agrupar por work_id para encontrar coautores reais
for _, group in train_df.groupby('work_id'):
    authors = group['author_id'].tolist()
    if len(authors) > 1:
        for u, v in itertools.permutations(authors, 2):
            grafo_treino[u].add(v)

print(f"Grafo construído com {len(grafo_treino)} autores.")

def gerar_candidatos_topologia(author_id, k=50):
    """Retorna até K candidatos baseados em amigos em comum (2 hops)."""
    if author_id not in grafo_treino:
        return []
        
    vizinhos = grafo_treino[author_id]
    candidatos_counter = Counter()
    
    for vizinho in vizinhos:
        vizinhos_do_vizinho = grafo_treino.get(vizinho, set())
        for candidato in vizinhos_do_vizinho:
            # Não recomendar a si mesmo, e não recomendar quem já é coautor
            if candidato != author_id and candidato not in vizinhos:
                candidatos_counter[candidato] += 1
                
    # Retorna os top K ordenados pela quantidade de amigos em comum
    return [c[0] for c in candidatos_counter.most_common(k)]

# Testando a geração de candidatos para ver se está funcionando
if autores_alvo:
    autor_teste = list(autores_alvo)[0]
    cands = gerar_candidatos_topologia(autor_teste, k=5)
    print(f"Teste de Topologia para {autor_teste} -> {len(cands)} candidatos encontrados.")

Construindo o grafo de coautorias do Treino (Passado)...
Grafo construído com 25260 autores.
Teste de Topologia para https://openalex.org/A5052325109 -> 5 candidatos encontrados.


In [10]:
print("Carregando bases para extração de features semânticas e de metadados...")
concepts_df = pd.read_csv(f'{database_path}/concepts.csv')

# 1. Mapeamento Autor -> Tópicos
print("Mapeando Tópicos...")
train_authors_works = train_df[['work_id', 'author_id']]
author_topic_merge = train_authors_works.merge(topics_df[['work_id', 'topic_id']], on='work_id')

mapa_topicos = defaultdict(set)
for _, row in author_topic_merge.iterrows():
    mapa_topicos[row['author_id']].add(row['topic_id'])
    
# 2. Mapeamento Autor -> Conceitos (Limitando a Level 0 e 1 para focar no macro)
print("Mapeando Conceitos...")
concepts_filtrados = concepts_df[concepts_df['level'].isin([0, 1])]
author_concept_merge = train_authors_works.merge(concepts_filtrados[['work_id', 'concept_id']], on='work_id')

mapa_conceitos = defaultdict(set)
for _, row in author_concept_merge.iterrows():
    mapa_conceitos[row['author_id']].add(row['concept_id'])

# 3. Mapeamento Autor -> País e Instituição (múltiplos valores separados por "|")
print("Mapeando Países e Instituições...")
mapa_pais = defaultdict(set)
mapa_instituicao = defaultdict(set)

for _, row in train_df.iterrows():
    author_id = row['author_id']
    
    # Países: separados por "|"
    if pd.notna(row.get('countries')):
        countries_str = str(row['countries']).strip("[]'\"")
        if countries_str:
            # Separar por "|" e limpar cada país
            countries = [c.strip() for c in countries_str.split('|') if c.strip()]
            mapa_pais[author_id].update(countries)
    
    # Instituições: separadas por "|"
    if pd.notna(row.get('institution_ids')):
        institutions_str = str(row['institution_ids']).strip("[]'\"")
        if institutions_str:
            # Separar por "|" e limpar cada instituição
            institutions = [i.strip() for i in institutions_str.split('|') if i.strip()]
            mapa_instituicao[author_id].update(institutions)

print("Bases de conhecimento criadas com sucesso!")

Carregando bases para extração de features semânticas e de metadados...
Mapeando Tópicos...
Mapeando Conceitos...
Mapeando Países e Instituições...
Bases de conhecimento criadas com sucesso!


In [11]:
print("Gerando dataset de treino (Pares Positivos vs Negativos Difíceis)...")

dataset_treino = []
QTD_PARES = 5000 # Começando com 5k de cada para não demorar, depois podemos aumentar

# 1. PARES POSITIVOS: Autores que colaboraram no treino
obras_amostra = train_df['work_id'].unique()[:3000] 
pares_positivos = set()

for work in obras_amostra:
    autores_obra = train_df[train_df['work_id'] == work]['author_id'].tolist()
    if len(autores_obra) > 1:
        for u, v in itertools.combinations(autores_obra, 2):
            par = tuple(sorted([u, v])) # Garante a ordem (A, B)
            pares_positivos.add(par)
            if len(pares_positivos) >= QTD_PARES:
                break
    if len(pares_positivos) >= QTD_PARES:
        break

# 2. PARES NEGATIVOS DIFÍCEIS (Hard Negatives)
# Vamos usar a topologia para encontrar pessoas com amigos em comum, mas que não colaboraram
pares_negativos = set()
autores_list = list(grafo_treino.keys())

for u in autores_list[:3000]: 
    candidatos = gerar_candidatos_topologia(u, k=10)
    for v in candidatos:
        par = tuple(sorted([u, v]))
        if par not in pares_positivos:
            pares_negativos.add(par)
        if len(pares_negativos) >= QTD_PARES: # Balanceamento 50/50
            break
    if len(pares_negativos) >= QTD_PARES:
        break

# 3. EXTRAÇÃO DAS FEATURES
def extrair_features(u, v, label):
    # Topologia
    vizinhos_u = grafo_treino.get(u, set())
    vizinhos_v = grafo_treino.get(v, set())
    common_neighbors = len(vizinhos_u.intersection(vizinhos_v))
    
    # Semântica
    topic_overlap = len(mapa_topicos.get(u, set()).intersection(mapa_topicos.get(v, set())))
    concept_overlap = len(mapa_conceitos.get(u, set()).intersection(mapa_conceitos.get(v, set())))
    
    # Metadados (verificando interseção de sets, já que autores podem ter múltiplos países/instituições)
    paises_u = mapa_pais.get(u, set())
    paises_v = mapa_pais.get(v, set())
    same_country = 1 if paises_u and paises_v and len(paises_u.intersection(paises_v)) > 0 else 0
    
    instituicoes_u = mapa_instituicao.get(u, set())
    instituicoes_v = mapa_instituicao.get(v, set())
    same_institution = 1 if instituicoes_u and instituicoes_v and len(instituicoes_u.intersection(instituicoes_v)) > 0 else 0
    
    return {
        'autor_A': u,
        'autor_B': v,
        'common_neighbors': common_neighbors,
        'topic_overlap': topic_overlap,
        'concept_overlap': concept_overlap,
        'same_country': same_country,
        'same_institution': same_institution,
        'label': label
    }

print("Calculando features para todos os pares...")
for u, v in pares_positivos:
    dataset_treino.append(extrair_features(u, v, 1))
    
for u, v in pares_negativos:
    dataset_treino.append(extrair_features(u, v, 0))

df_ml = pd.DataFrame(dataset_treino)
print(f"Dataset de Machine Learning criado com sucesso: {len(df_ml)} linhas!")
display(df_ml.head())

Gerando dataset de treino (Pares Positivos vs Negativos Difíceis)...
Calculando features para todos os pares...
Dataset de Machine Learning criado com sucesso: 10000 linhas!


,autor_A,autor_B,common_neighbors,topic_overlap,concept_overlap,same_country,same_institution,label
0,https://openalex.org/A5025478860,https://openalex.org/A5103147587,14,4,4,1,0,1
1,https://openalex.org/A5041572754,https://openalex.org/A5102099783,8,3,4,1,0,1
2,https://openalex.org/A5005545405,https://openalex.org/A5078669013,3,6,50,1,1,1
3,https://openalex.org/A5031646349,https://openalex.org/A5074649269,4,2,6,0,0,1
4,https://openalex.org/A5003869173,https://openalex.org/A5005545405,13,3,10,0,0,1


In [12]:
# =================================================================
# CÉLULA 5: AVALIAÇÃO DE COMBINAÇÕES E CÁLCULO DE MRR
# =================================================================
from sklearn.ensemble import RandomForestClassifier
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

print("Preparando dados de teste (Futuro)...")
# 1. Pegar os verdadeiros coautores do futuro para os nossos autores alvo
true_future_links = defaultdict(set)
for _, group in test_df.groupby('work_id'):
    authors = group['author_id'].tolist()
    if len(authors) > 1:
        for u, v in itertools.permutations(authors, 2):
            if u in autores_alvo:
                true_future_links[u].add(v)

# Vamos amostrar 100 autores alvo que tiveram coautores novos no futuro para avaliar rápido
autores_avaliar = [u for u, v in true_future_links.items() if len(v) > 0][:100]
print(f"Avaliando MRR para {len(autores_avaliar)} autores alvo...\n")

# Todas as features que criamos
features_disponiveis = ['common_neighbors', 'topic_overlap', 'concept_overlap', 'same_country', 'same_institution']

def calcular_mrr(recomendacoes_dit, verdadeiros_links_dit, k=10):
    mrr_soma = 0.0
    for autor, preds in recomendacoes_dit.items():
        verdadeiros = verdadeiros_links_dit.get(autor, set())
        for rank, p in enumerate(preds[:k], 1):
            if p in verdadeiros:
                mrr_soma += 1.0 / rank
                break
    return mrr_soma / len(recomendacoes_dit) if recomendacoes_dit else 0

def testar_combinacao(features_teste):
    # 1. Treinar o modelo apenas com as features selecionadas
    X_train = df_ml[features_teste]
    y_train = df_ml['label']
    
    clf = RandomForestClassifier(n_estimators=50, max_depth=5, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)
    
    # 2. Re-ranquear os candidatos para os autores de teste
    recomendacoes_finais = {}
    
    for autor in autores_avaliar:
        # A Topologia gera os Top 50 candidatos (Recall alto)
        candidatos = gerar_candidatos_topologia(autor, k=50)
        
        if not candidatos:
            recomendacoes_finais[autor] = []
            continue
            
        # Extrair features dinamicamente para cada candidato
        features_candidatos = []
        for cand in candidatos:
            feat_dict = extrair_features(autor, cand, 0) # Label não importa aqui
            features_candidatos.append([feat_dict[f] for f in features_teste])
            
        # Predizer a probabilidade (Classe 1 = Vai colaborar)
        probs = clf.predict_proba(features_candidatos)[:, 1]
        
        # Juntar candidatos com probabilidades e ordenar (Re-ranqueamento)
        candidatos_rankeados = [c for _, c in sorted(zip(probs, candidatos), reverse=True)]
        recomendacoes_finais[autor] = candidatos_rankeados
        
    # 3. Calcular MRR
    return calcular_mrr(recomendacoes_finais, true_future_links, k=10)

# =================================================================
# LOOP DE BUSCA (A MÁGICA ACONTECE AQUI)
# =================================================================
resultados = []
# Vamos testar combinações de 2 a 5 features (sempre incluindo topologia)
for r in range(2, len(features_disponiveis) + 1):
    for combo in combinations(features_disponiveis, r):
        combo_list = list(combo)
        
        # A topologia (common_neighbors) é a base de tudo, ela tem que estar na combinação
        if 'common_neighbors' not in combo_list:
            continue
            
        # print(f"Testando: {combo_list}...")
        mrr = testar_combinacao(combo_list)
        resultados.append({'Features': " + ".join(combo_list), 'MRR@10': mrr})

# Ordenar e mostrar os resultados
df_resultados = pd.DataFrame(resultados).sort_values(by='MRR@10', ascending=False)

print("\n" + "="*60)
print("🏆 RANKING DAS MELHORES COMBINAÇÕES DE FEATURES 🏆")
print("="*60)
display(df_resultados.head(10))

# Ver o desempenho só da topologia pura para comparar
mrr_topologia_pura = testar_combinacao(['common_neighbors'])
print(f"\nBaseline (Apenas Topologia): MRR@10 = {mrr_topologia_pura:.4f}")
print(f"Melhor Híbrido: MRR@10 = {df_resultados.iloc[0]['MRR@10']:.4f}")
print(f"Melhora de: {((df_resultados.iloc[0]['MRR@10'] - mrr_topologia_pura) / (mrr_topologia_pura+0.0001)) * 100:.2f}%")

Preparando dados de teste (Futuro)...
Avaliando MRR para 100 autores alvo...


🏆 RANKING DAS MELHORES COMBINAÇÕES DE FEATURES 🏆


,Features,MRR@10
1,common_neighbors + concept_overlap,0.290111
11,common_neighbors + topic_overlap + concept_ove...,0.283444
4,common_neighbors + topic_overlap + concept_ove...,0.275417
8,common_neighbors + concept_overlap + same_inst...,0.266111
13,common_neighbors + concept_overlap + same_coun...,0.235623
10,common_neighbors + topic_overlap + concept_ove...,0.203040
7,common_neighbors + concept_overlap + same_country,0.201468
14,common_neighbors + topic_overlap + concept_ove...,0.177762
0,common_neighbors + topic_overlap,0.162556
2,common_neighbors + same_country,0.144444



Baseline (Apenas Topologia): MRR@10 = 0.1618
Melhor Híbrido: MRR@10 = 0.2901
Melhora de: 79.22%


In [13]:
df_resultados

,Features,MRR@10
1,common_neighbors + concept_overlap,0.290111
11,common_neighbors + topic_overlap + concept_ove...,0.283444
4,common_neighbors + topic_overlap + concept_ove...,0.275417
8,common_neighbors + concept_overlap + same_inst...,0.266111
13,common_neighbors + concept_overlap + same_coun...,0.235623
10,common_neighbors + topic_overlap + concept_ove...,0.203040
7,common_neighbors + concept_overlap + same_country,0.201468
14,common_neighbors + topic_overlap + concept_ove...,0.177762
0,common_neighbors + topic_overlap,0.162556
2,common_neighbors + same_country,0.144444
